In [1]:
import pathpyG as pp
import torch
from torch_geometric.utils import cumsum, coalesce, degree, sort_edge_index

from tqdm import tqdm

In [2]:
# t_sp = pp.TemporalGraph.from_csv('sociopatterns_highschool_2013.tedges').to_undirected()
# print(t_sp)
# print(torch.unique(t_sp.data.t).size(0))

In [3]:
t = pp.TemporalGraph.from_edge_list([(0,1,0), (0,2,0), (1,2,1), (1,3,1), (3,4,2)])
print(t)

Temporal Graph with 5 nodes, 5 unique edges and 5 events in [0.0, 2.0]

Graph attributes
	src		<class 'torch.Tensor'> -> torch.Size([5])
	t		<class 'torch.Tensor'> -> torch.Size([5])
	dst		<class 'torch.Tensor'> -> torch.Size([5])



/opt/conda/lib/python3.10/site-packages/torch_geometric/data/storage.py:450: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'src', 't', 'dst'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


In [4]:
# new memory-efficient code copied from `temporal_shortest_paths.ipynb`
def lift_order_efficient(g: pp.TemporalGraph, delta: int = 1):

    # first-order edge index
    edge_index, timestamps = g.data.edge_index, g.data.t

    #print(edge_index)
    #print(timestamps)

    indices = torch.arange(0, edge_index.size(1), device=g.data.edge_index.device)

    unique_t, reverse_idx = torch.unique(timestamps, sorted=True, return_inverse=True)
    second_order = []
    count = 0

    # lift order: find possible continuations for edges in each time stamp
    for i in tqdm(range(unique_t.size(0))):
        t = unique_t[i]
        #print('timestamp index ', i)
        #print('timestamp ', t)
        
        # find indices of all source edges that occur at unique timestamp t
        src_time_mask = (timestamps == t)
        src_edges = edge_index[:,src_time_mask]
        src_edge_idx = indices[src_time_mask]
        #print(src_edges)
        #print(src_edge_idx)

        # find indices of all edges that can continue edges at tine t for given delta
        dst_time_mask = (timestamps > t) & (timestamps <= t+delta)
        dst_edges = edge_index[:,dst_time_mask]        
        dst_edge_idx = indices[dst_time_mask]
        #print(dst_edges)
        #print(dst_edge_idx)

        if dst_edge_idx.size(0)>0 and src_edge_idx.size(0)>0:

            # compute second-order edges between src and dst idx for all edges where dst in src_edges matches src in dst_edges        
            x = torch.cartesian_prod(src_edge_idx, dst_edge_idx).t()
            src_edges = torch.index_select(edge_index, dim=1, index=x[0])
            dst_edges = torch.index_select(edge_index, dim=1, index=x[1])
            #print(src_edges)
            #print(dst_edges)
            ho_edge_index = x[:,torch.where(src_edges[1,:] == dst_edges[0,:])[0]]
            second_order.append(ho_edge_index)
            #print(ho_edge_index) 
            
            # #print('dst', dst)
            # src_mask = (edge_index[:,mask][0]==dst)
            # ctd = edge_index[:,mask][:,src_mask]
            # #print('continuations', ctd)
            # ctd_indices = torch.where(edge_index[:,mask][0]==dst)[0]        
            # #print('ctd indx', ctd_indices)
            # count += ctd_indices.size(0)
    ho_index = torch.cat(second_order, dim=1)    
    return ho_index.size(1), ho_index

In [5]:
def time_respecting_paths(g: pp.TemporalGraph, delta: int) -> dict:
    """
    Calculate all longest time-respecting paths in a temporal graph.
    """
    paths_of_length = {}

    node_sequence = torch.arange(g.data.num_nodes, device=g.data.edge_index.device).unsqueeze(1)
    node_sequence = torch.cat([node_sequence[g.data.edge_index[0]], node_sequence[g.data.edge_index[1]][:, -1:]], dim=1)
    edge_index = lift_order_efficient(g, delta)[1]
    
    # calculate degrees
    out_degree = degree(edge_index[0], num_nodes=g.M, dtype=torch.long)
    in_degree = degree(edge_index[1], num_nodes=g.M, dtype=torch.long)
    # identify root nodes with in-degree zero
    roots = torch.where(in_degree == 0)[0]
    leafs = (out_degree == 0)
    # print("Roots:", roots)
    # print("Leafs:", leafs)
    paths = node_sequence[roots]
    paths_of_length[1] = paths[leafs[roots]].cpu()

    paths = paths[~leafs[roots]]
    nodes = roots[~leafs[roots]]

    ptrs = cumsum(out_degree, dim=0)


    # count all longest time-respecting paths in the temporal graph
    step = 1
    while nodes.size(0) > 0:
        # print("step", step)
        # print("Paths: ", paths)
        # print("Nodes: ", nodes)
        idx_repeat = torch.repeat_interleave(out_degree[nodes])
        next_idx = torch.repeat_interleave(ptrs[nodes], out_degree[nodes])
        idx_correction = torch.arange(next_idx.size(0), device=edge_index.device) - cumsum(out_degree[nodes], dim=0)[idx_repeat]
        next_idx += idx_correction
        next_nodes = edge_index[1][next_idx]
        paths = torch.cat([paths[idx_repeat], node_sequence[next_nodes, 1:]], dim=1)
        paths_of_length[step] = paths[leafs[next_nodes]].tolist()
        paths = paths[~leafs[next_nodes]]
        nodes = next_nodes[~leafs[next_nodes]]
        step += 1

    return paths_of_length


In [6]:
lift_order_efficient(t, delta=1)

100%|██████████| 3/3 [00:00<00:00, 1277.45it/s]


(3,
 tensor([[0, 0, 3],
         [2, 3, 4]]))

In [7]:
# lift_order_efficient(t_sp, delta=300)

In [11]:
t.data.edge_index, t.data.t

(tensor([[0, 0, 1, 1, 3],
         [1, 2, 2, 3, 4]]),
 tensor([0., 0., 1., 1., 2.]))

In [8]:
time_respecting_paths(t, delta=1)

100%|██████████| 3/3 [00:00<00:00, 1443.49it/s]


{1: [[0, 1, 2]], 2: [[0, 1, 3, 4]]}